In [1]:
import pandas as pd
import numpy as np
import numpy.ma as ma 
import random 

pfps50 = pd.read_excel("Dane_PFSP_50_20.xlsx", index_col=[0])
pfps100 = pd.read_excel("Dane_PFSP_100_10.xlsx", index_col=[0])
pfps200 = pd.read_excel("Dane_PFSP_200_10.xlsx", index_col=[0])


def df_to_array(data):
  tasks = list(range(0, len(data)))
  times = data
  times = np.array(times)
  return times, tasks

def changeIndices(indices):
  return list(map(lambda x: x+1, indices))

In [2]:
def randomSolution(times, tasks):
    solution = []
    task_number_random = tasks.copy()
    for i in range(len(tasks)):
        random_step = task_number_random[random.randint(0,len(task_number_random)-1)]
        solution.append(random_step)
        task_number_random.remove(random_step)
    return solution

In [3]:
def generateCompletedTime(times, tasks):
  completedTime = times.copy()
  machinecCount = len(times[0])
  # pierwszy wiersz
   # j - index kolumny, i - index wiersza
  completedTime[0][0] = times[tasks[0]][0]
  i = 0
  for j in range(1, machinecCount):
    completedTime[i][j] = completedTime[i][j-1] + times[tasks[i]][j]
  # pierwsza kolumna
  # j - index kolumny, i - index wiersza
  j = 0
  for i in range(1, len(tasks)):
    completedTime[i][j] = completedTime[i-1][j] + times[tasks[i]][j]
  # środek
  # j - index kolumny, i - index wiersza
  for i in range(1, len(tasks)):
    for j in range(1, machinecCount):
      maxi = max(completedTime[i-1][j], completedTime[i][j-1])
      completedTime[i][j] = maxi + times[tasks[i]][j]
  return completedTime[len(tasks)-1][machinecCount-1]


In [4]:
def randomSolution_max(times, tasks,max_distance):
    solution = []
    tasks_number_random = tasks.copy()
    for i in range(len(tasks)):
        random_step = tasks_number_random[random.randint(0,len(tasks_number_random)-1)]
        solution.append(random_step)
        tasks_number_random.remove(random_step)
    if (generateCompletedTime(times, solution)>max_distance):
        solution = randomSolution_max(times, tasks,max_distance)
    return solution

In [5]:
times,tasks = df_to_array(pfps50)
randomSolution_max(times,tasks,4450)

[30,
 11,
 44,
 16,
 47,
 13,
 49,
 46,
 4,
 10,
 9,
 43,
 38,
 22,
 26,
 29,
 21,
 34,
 23,
 6,
 42,
 5,
 17,
 40,
 32,
 20,
 7,
 25,
 8,
 2,
 36,
 19,
 33,
 28,
 1,
 31,
 12,
 48,
 27,
 14,
 24,
 3,
 15,
 41,
 35,
 18,
 45,
 39,
 37,
 0]

In [6]:
def Neighbours(solution):
    neighbours = []
    for i in range(0,len(solution)):
        for j in range(i+1,len(solution)):
            neighbour = solution.copy()
            neighbour[i] =  solution[j]
            neighbour[j] = solution[i] 
            neighbours.append(neighbour) #lista z wszystkimi sąsiadami od podanego _tour_. Sąsiedztwo typ [1,2,3] -> [2,1,3],[3,2,1][1,3,2](wymiani kązdy zkażdym)
    return neighbours


In [7]:
def Neighbours2(solution): 
    neighbours = []
    for i in range(0,len(solution)):
        for j in range(0,len(solution)):
            neighbour = solution.copy()
            step = neighbour.pop(i)
            neighbour.insert(j,step)
            neighbours.append(neighbour) #lista z wszystkimi sąsiadami od podanego _tour_. Sąsiedztwo typ [1,2,3] -> [2,1,3],[2,3,1][1,3,2](wymiani kązdy zkażdym)
    return neighbours

In [8]:
for i in range(0,len([1,2,3])):
    print(i)

0
1
2


In [9]:
def InvertNeighbours(solution):
    neighbours = []
    for i in range(0,len(solution)):
        for j in range(0,len(solution)):
            if i==j:
                continue
            neighbour = []
            index1 = i
            index2 = j
            if index1 > index2:
                helper = index1
                index1 = index2
                index2 = helper
            x = 0
            j = 0
            while x < index1:
                neighbour.append(solution[x])
                x = x + 1
            x = index2
            while x >= index1:
                neighbour.append(solution[x])
                x = x - 1
            x = index2 + 1
            while x <= len(solution) - 1:
                neighbour.append(solution[x])
                x = x + 1
            neighbours.append(neighbour)
    return neighbours
        

In [10]:
def TheBestNeighbour2(times, tasks,neighbours,solution_distance): #metoda dla pierwszego lepszego sąsiada. wymiana [1,2,3,4] -> [3,2,1,4] swap wymiana 2 miast miejscami
    best_distance = solution_distance
    best_neighbour = []
    neighbours_method = neighbours.copy()
    while (len(neighbours_method) != 0):
        random_neighbour = random.choice(neighbours_method)
        distance = generateCompletedTime(times,random_neighbour)
        if distance < best_distance:
            best_distance = distance
            best_neighbour = random_neighbour.copy()
            break
        else:
            neighbours_method.remove(random_neighbour)
    return best_distance,best_neighbour


In [11]:
def IHC(xlsx_data,limit_for_itteration,kind_of_neighbours,max_start_distance):  #wybiera najlepszego sąsiada
    times,tasks = df_to_array(xlsx_data)
    random_solution = randomSolution_max(times,tasks,max_start_distance)
    solution_tour = random_solution.copy()
    completed_time =generateCompletedTime(times,random_solution)
    solution_distance = completed_time.copy()
    neighbours = kind_of_neighbours(random_solution)
    best_distance,best_neighbour = TheBestNeighbour2(times,tasks,neighbours,solution_distance)
    number_of_iterration = 0

    while ((solution_distance > best_distance) and (number_of_iterration < limit_for_itteration) ): #jeśli zaszła zmiana najlepszego wyniku (nie byliśmy w ekstremum.)
        solution_tour = best_neighbour
        solution_distance = best_distance
        neighbours = kind_of_neighbours(solution_tour)
        best_distance,best_neighbour = TheBestNeighbour2(times,tasks,neighbours,solution_distance)
        number_of_iterration += 1 


    return solution_tour,solution_distance,random_solution,completed_time,number_of_iterration


    

In [24]:
def TheBestIHC(number_start,xlsx_data,limit,kind_of_neighbours,max_start_distance):
    best_soltution,best_result,_,_,iteration_number= IHC(xlsx_data,limit,kind_of_neighbours,max_start_distance)
    sum_avg = best_result
    best_tour_show = changeIndices(best_soltution)
    for i in range(0,number_start):
        solution_tour,solution_distance,random_tour,tour_distance,number_of_iterration= IHC(xlsx_data,limit,kind_of_neighbours,max_start_distance)
        sum_avg += solution_distance 
        if (best_result > solution_distance) :
            best_result = solution_distance
            best_soltution = solution_tour
            iteration_number = number_of_iterration
            best_tour_show = changeIndices(best_soltution)
            print(f"""
            wynik : {best_result} , liczba iteracji wewnątrz algorytmu:{iteration_number}
            trasa: {best_tour_show}
            """) 
            result_data = {'best_result': best_result, 'best_tour_show':best_tour_show, 'iteration_number':iteration_number, 'number_start':number_start,'limit':limit,'kind_of_neighbours':kind_of_neighbours,'max_start_distance':max_start_distance}
            df = pd.DataFrame(result_data, columns=['best_result', 'best_tour_show','iteration_number','number_start','limit','kind_of_neighbours','max_start_distance'])
            df.to_excel("best_result.xlsx")
    return best_result,best_tour_show,iteration_number,(sum_avg/(number_start+1))

In [13]:
best_result,best_solution,iteration_number,avg = TheBestIHC(2,pfps50,200,Neighbours2,4450)
print(f"Najlepszy wynik : {best_result} \n Najlepsza trasa : {best_solution}\n Liczba iteracji w najlepszej próbie : {iteration_number},\n średnia : {round(avg,2)} ")


KeyboardInterrupt: 

In [ ]:
start_iteration = [20,100,1000,3000]
ihc_iteration = [200]
kind_neighbours = [Neighbours,Neighbours2]
start_distance = [4400,4600,4800,5200]

for kn in kind_neighbours:
    for sd in start_distance:
        best_result,best_solution,iteration_number,avg = TheBestIHC(99,pfps50,200,kn,sd)
        print(f"Najlepszy wynik : {best_result} \n Najlepsza trasa : {best_solution}\n Liczba iteracji w najlepszej próbie : {iteration_number},\n średnia : {round(avg,2)} \n 200 \n {kn.__name__} \n {sd} ")



Najlepszy wynik : 3796 
 Najlepsza trasa : [24, 28, 22, 3, 46, 5, 18, 19, 31, 10, 47, 23, 16, 33, 43, 30, 8, 15, 39, 27, 32, 17, 4, 9, 20, 11, 35, 6, 36, 49, 48, 45, 21, 1, 40, 2, 50, 14, 41, 37, 12, 13, 26, 38, 25, 44, 34, 29, 42, 7]
 Liczba iteracji w najlepszej próbie : 55,
 średnia : 3898.09 
 200 
 Neighbours 
 4400 
Najlepszy wynik : 3811 
 Najlepsza trasa : [27, 24, 19, 46, 39, 2, 8, 32, 4, 48, 28, 45, 6, 16, 36, 29, 25, 3, 21, 18, 5, 43, 37, 9, 11, 20, 22, 40, 30, 15, 26, 47, 33, 49, 35, 42, 10, 31, 23, 17, 12, 44, 1, 50, 38, 13, 14, 41, 7, 34]
 Liczba iteracji w najlepszej próbie : 68,
 średnia : 3911.1 
 200 
 Neighbours 
 4600 
Najlepszy wynik : 3818 
 Najlepsza trasa : [24, 19, 46, 39, 48, 8, 31, 12, 16, 22, 15, 9, 4, 27, 5, 45, 17, 2, 29, 33, 43, 21, 37, 32, 11, 10, 26, 1, 3, 30, 20, 50, 18, 40, 35, 6, 47, 49, 25, 23, 34, 36, 42, 28, 41, 7, 13, 44, 14, 38]
 Liczba iteracji w najlepszej próbie : 69,
 średnia : 3908.99 
 200 
 Neighbours 
 4800 
Najlepszy wynik : 3806 
 Najl

In [22]:
ihc_iteration = [20,50,100,200]
kind_neighbours = [Neighbours,Neighbours2,InvertNeighbours]
start_distance = [5600,6000]

for kn in kind_neighbours:
    for ihc in ihc_iteration:
        for sd in start_distance:
            best_result,best_solution,iteration_number,avg = TheBestIHC(99,pfps50,ihc,kn,sd)
            print(f"Najlepszy wynik : {best_result} \n Najlepsza trasa : {best_solution}\n Liczba iteracji w najlepszej próbie : {iteration_number},\n średnia : {round(avg,2)} \n {ihc} \n {kn.__name__} \n {sd} \n ---------------------------- ")


Najlepszy wynik : 4082 
 Najlepsza trasa : [27, 32, 25, 39, 41, 8, 15, 45, 49, 43, 16, 28, 19, 23, 35, 37, 1, 11, 22, 4, 47, 5, 17, 36, 9, 48, 34, 50, 21, 18, 29, 14, 40, 30, 3, 2, 12, 20, 31, 7, 46, 24, 44, 33, 26, 6, 13, 42, 38, 10]
 Liczba iteracji w najlepszej próbie : 20,
 średnia : 4234.46 
 20 
 Neighbours 
 5600 
 ---------------------------- 
Najlepszy wynik : 4058 
 Najlepsza trasa : [39, 12, 24, 8, 16, 27, 32, 3, 46, 31, 14, 11, 10, 26, 49, 23, 43, 18, 19, 29, 13, 17, 45, 21, 15, 41, 28, 22, 37, 33, 48, 50, 30, 9, 7, 34, 1, 5, 47, 4, 6, 20, 35, 2, 36, 40, 42, 44, 25, 38]
 Liczba iteracji w najlepszej próbie : 20,
 średnia : 4228.91 
 20 
 Neighbours 
 6000 
 ---------------------------- 
Najlepszy wynik : 3864 
 Najlepsza trasa : [24, 22, 8, 31, 10, 15, 28, 19, 6, 48, 45, 46, 47, 11, 20, 7, 16, 2, 12, 5, 23, 25, 9, 37, 18, 4, 50, 30, 49, 42, 32, 39, 35, 41, 17, 3, 44, 34, 29, 14, 21, 43, 27, 40, 26, 36, 1, 13, 38, 33]
 Liczba iteracji w najlepszej próbie : 50,
 średnia : 396

In [28]:
TheBestIHC(199,pfps50,200,Neighbours2,4400)


            wynik : 3844 , liczba iteracji wewnątrz algorytmu:69
            trasa: [24, 46, 10, 41, 16, 3, 25, 28, 35, 39, 49, 2, 37, 47, 11, 23, 22, 9, 43, 1, 8, 48, 19, 15, 50, 27, 26, 30, 33, 21, 32, 45, 34, 6, 5, 20, 4, 36, 17, 29, 14, 38, 42, 31, 12, 18, 40, 44, 13, 7]
            

            wynik : 3757 , liczba iteracji wewnątrz algorytmu:77
            trasa: [24, 4, 28, 46, 39, 22, 17, 11, 41, 5, 49, 8, 16, 43, 15, 45, 27, 2, 29, 31, 14, 23, 32, 33, 20, 37, 48, 50, 34, 6, 36, 35, 25, 47, 12, 18, 3, 19, 42, 21, 30, 9, 26, 38, 1, 13, 10, 40, 44, 7]
            

            wynik : 3753 , liczba iteracji wewnątrz algorytmu:71
            trasa: [24, 4, 28, 39, 8, 31, 16, 22, 5, 32, 11, 46, 25, 12, 14, 18, 2, 15, 47, 9, 20, 38, 19, 21, 30, 49, 17, 43, 33, 41, 37, 6, 48, 26, 35, 27, 36, 29, 3, 34, 23, 40, 13, 42, 10, 45, 50, 44, 1, 7]
            


KeyboardInterrupt: 

In [26]:
TheBestIHC(99,pfps100,200,Neighbours2,6400)


            wynik : 5574 , liczba iteracji wewnątrz algorytmu:109
            trasa: [79, 65, 56, 83, 33, 90, 30, 27, 100, 77, 84, 37, 70, 40, 97, 73, 13, 6, 54, 76, 58, 22, 48, 92, 87, 23, 41, 14, 9, 61, 47, 60, 35, 81, 5, 51, 72, 88, 10, 4, 55, 75, 42, 94, 93, 71, 74, 45, 89, 98, 2, 50, 17, 99, 69, 16, 63, 68, 34, 49, 39, 44, 18, 66, 57, 96, 62, 12, 28, 80, 91, 25, 15, 82, 46, 95, 85, 3, 53, 7, 20, 64, 21, 11, 31, 38, 29, 43, 1, 36, 8, 86, 52, 67, 24, 19, 78, 26, 59, 32]
            

            wynik : 5567 , liczba iteracji wewnątrz algorytmu:106
            trasa: [75, 56, 79, 83, 48, 97, 10, 94, 99, 45, 38, 21, 28, 73, 23, 68, 57, 85, 55, 7, 77, 95, 70, 51, 39, 17, 27, 3, 81, 44, 2, 47, 9, 24, 1, 86, 92, 12, 32, 4, 16, 74, 66, 19, 62, 58, 100, 89, 61, 34, 31, 46, 98, 25, 50, 64, 76, 5, 42, 14, 80, 90, 22, 30, 54, 96, 72, 60, 33, 15, 71, 35, 13, 20, 78, 69, 8, 40, 88, 36, 91, 11, 41, 84, 93, 37, 65, 63, 53, 67, 87, 29, 43, 6, 52, 18, 59, 49, 26, 82]
            

            wyn

(5559,
 [79,
  83,
  56,
  97,
  10,
  45,
  1,
  48,
  90,
  27,
  4,
  47,
  36,
  37,
  61,
  34,
  2,
  30,
  20,
  7,
  73,
  81,
  72,
  17,
  92,
  55,
  5,
  80,
  42,
  74,
  69,
  88,
  85,
  21,
  98,
  11,
  46,
  33,
  16,
  57,
  63,
  95,
  65,
  12,
  89,
  32,
  58,
  87,
  44,
  49,
  35,
  31,
  75,
  100,
  28,
  76,
  39,
  96,
  22,
  67,
  14,
  23,
  9,
  41,
  8,
  38,
  18,
  13,
  24,
  62,
  54,
  60,
  6,
  64,
  84,
  15,
  70,
  71,
  43,
  50,
  52,
  51,
  19,
  77,
  82,
  94,
  66,
  93,
  68,
  99,
  3,
  40,
  86,
  25,
  29,
  59,
  53,
  91,
  26,
  78],
 117,
 5644.96)

In [27]:
TheBestIHC(49,pfps200,300,Neighbours2,11750)


            wynik : 10622 , liczba iteracji wewnątrz algorytmu:167
            trasa: [22, 97, 184, 58, 9, 59, 156, 107, 191, 25, 16, 143, 167, 91, 152, 68, 171, 42, 158, 179, 64, 80, 2, 185, 47, 87, 155, 102, 85, 176, 153, 35, 28, 137, 103, 162, 108, 27, 12, 173, 84, 200, 89, 82, 199, 11, 192, 94, 52, 113, 67, 134, 20, 163, 146, 43, 139, 168, 86, 117, 32, 178, 17, 48, 175, 5, 140, 106, 122, 54, 182, 3, 56, 197, 165, 31, 14, 125, 19, 6, 120, 127, 135, 181, 62, 169, 104, 90, 33, 4, 133, 39, 44, 170, 40, 116, 15, 45, 189, 55, 63, 70, 29, 198, 75, 93, 71, 161, 128, 50, 194, 154, 118, 144, 130, 124, 172, 77, 123, 72, 180, 166, 99, 183, 136, 119, 37, 114, 174, 148, 34, 160, 88, 13, 150, 26, 95, 105, 98, 188, 126, 1, 76, 41, 24, 131, 164, 65, 96, 141, 142, 159, 66, 78, 57, 61, 112, 190, 18, 92, 83, 49, 100, 115, 132, 81, 74, 7, 177, 21, 145, 110, 51, 30, 23, 36, 46, 193, 149, 195, 69, 121, 187, 111, 157, 147, 38, 151, 109, 8, 73, 138, 186, 101, 129, 60, 196, 53, 79, 10]
            

      

KeyboardInterrupt: 